In [27]:
import numpy as np
from numpy import linalg as la
#####################################################################################
#################Explicit Finite Differences Method on S=exp(x)######################
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
#########################===========!The Function!==========#########################
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
#This function will return the FairValue of a Vanilla European Call/Put
# S_max: is a high Stock Price, that cannot be reached
# T:     is the maturity
# r:     is the risk free rate
# q:     is the dividend yield
# sigma: is the volatility of the stock
# N:     is the time discretization number
# M:     is the stock discretization parameter
# isCall:is the derivative type call or put
# isAmerican: Trivial
def VanillaOption(S_max,
                  T,
                  K,
                  r,
                  q,
                  sigma,
                  N,
                  M,
                  isCall,
                  isAmerican):
    outVector = np.zeros((2,1),dtype = float)
    #Discretization of the Stock and the Time
    dS=S_max/float(M)   
    dt=T/float(N)
    
    j=np.arange(1,M,dtype=np.float)   
    
    #'Transition Probabilities'
    a=(-0.5*(r-q)*dt*j+0.5*sigma**2*dt*j**2)/(1.0+r*dt)
    b=(1.0-sigma**2*dt*j**2)/(1.0+r*dt)
    c=(0.5*(r-q)*dt*j+0.5*sigma**2*dt*j**2)/(1.0+r*dt)
    #The matrix from the transition probabilities
    A=np.diag(b)+np.diag(a[1:],k=-1)+np.diag(c[0:M-2],k=1)
    infinityNormOfA = la.norm(A,np.inf)
    outVector[1] = infinityNormOfA
    #The matrix for the option price
    #N+1: Time Steps
    #M+1: Stock Steps
    FV=np.zeros((N+1,M+1))
    
    #Set up the boundary conditions
        #-> At time 0 the values are zero
        #-> The maximum value is S_max
    FV[:,0]=0
    FV[:,M]=S_max#[S_max * np.exp(-r*( N - j)*dt) for j in range(N+1)]
    if(bool(isCall)):
        FV[N,:]=np.maximum(np.arange(0,S_max+dS/2.0,dS,dtype=np.float)-K,0)
    else:
        FV[N,:]=np.maximum(K-np.arange(0,S_max+dS/2.0,dS,dtype=np.float),0)
    
    FV=np.matrix(np.array(FV))
    #Start the Backward Iteration
    for i in range(FV.shape[0]-2,-1,-1):
        k_i=np.zeros((M-1,1))
        #Inserts the first and the last element
        k_i[0]=a[0]*FV[i+1,0] 
        k_i[M-2]=c[M-2]*FV[i+1,M]
        
        FV[i,1:M]=np.dot(FV[i+1,1:M],A)+np.matrix(np.array(k_i)).transpose()
        
        if(bool(isAmerican)):
            if(bool(isCall)):
                FV[i,:] = np.maximum(np.arange(0,S_max+dS/2,dS,dtype=float)-K,FV[i,:]) 
            else:
                FV[i,:] = np.maximum(K-np.arange(0,S_max+dS/2,dS,dtype=float),FV[i,:])
        outVector[0] = FV[0,(FV.shape[1]+2)/2]
    return outVector
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
#########################========End Of The Function========#########################
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
S_max = 100.0
T=1.0
K=50.0
r=0.1
q=0.01
sigma = 0.25
N=200
M=50
isCall = 1
isAmerican = 0
print VanillaOption(S_max,
                    T,
                    K,
                    r,
                    q,
                    sigma,
                    N,
                    M,
                    isCall,
                    isAmerican)        


[[ 6.85590488]
 [ 0.99956897]]


In [185]:
import math as math
def phi(x):
    'Cumulative distribution function for the standard normal distribution'
    return (1.0 + math.erf(x / sqrt(2.0))) / 2.0
def BlackScholes(S,
                 T,
                 K,
                 r,
                 q,
                 sigma,
                 isCall):
    d1=(np.log(S/K)-0.5*T*(r-q))/(sigma*np.sqrt(T))+0.5*np.sqrt(T)*sigma
    d2=d1-np.sqrt(T)*sigma
    return S*phi(d1)- math.exp(-r*T)*K*phi(d2)
BlackScholes(50,1,50,0.1,0.01,0.25,0)



6.7032679441871466